<a href="https://colab.research.google.com/github/hholl19/Deep-Leaning-Class-Notes/blob/hholl19_added_things/Applied_Deep_Learning_2nd_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Snake Classifier Model Attempt 3**

**Hadassah Holl**

**Load Dependencies**

In [1]:
import keras
import pandas as pd
import numpy as np
import os
import pathlib 
from pathlib import Path
#import cv2
#from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten, Conv2D, MaxPooling2D 
from keras.layers.normalization import BatchNormalization 
from keras.optimizers import SGD
#from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg19 import VGG19

from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
#from matplotlib import pyplot

**Mounted Google Drive and wrote in Path directly to my Google Drive to Gain Access to full Dataset**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path= '/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs'
folders = os.listdir(file_path)
print(folders)

['test.csv', 'train.csv', 'train', 'test']


In [4]:
df_train= pd.read_csv('/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train.csv')
df_train.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


**Added in image file paths to pull images into CSV for easier processing.**

In [5]:
df_train["image_id"]="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/" + df_train["image_id"] + ".jpg"
df_train.head()

,image_id,breed
0,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,nerodia-erythrogaster
1,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,pantherophis-vulpinus
2,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,thamnophis-sirtalis
3,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,pantherophis-obsoletus
4,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,agkistrodon-contortrix


**Load Pre-trained VGG19 Model**

In [6]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

**Freeze the main model layers**

In [7]:
for layer in vgg19.layers:
    layer.trainable = False

*Started the training model, loaded in the pre-trained vgg19, and added some additional layers per class example.*

In [20]:
model = Sequential()
model.add(vgg19)
 
model.add(Flatten(name='flattened'))
#model.add(Dropout(0.3, name='dropout'))
model.add(Dense(35, activation='softmax', name='predictions'))

In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

**Adding Data Augmentation and Pre-processing together for Transfer Learning.**

In [22]:
train_datagen = ImageDataGenerator(rescale=1/255,
        validation_split=0.2,
        data_format='channels_last',
        rotation_range=30,
        horizontal_flip=True,
        fill_mode='reflect') 
test_datagen = ImageDataGenerator(
        rescale=1.0/255,
        data_format='channels_last')

In [23]:
train_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/",
        x_col="image_id",
        y_col="breed",
        subset="training",
        batch_size=32,
        seed=123,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", 
        target_size=(224,224)) 

Found 4407 validated image filenames belonging to 35 classes.


In [18]:
test_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/",
        x_col="image_id",
        y_col="breed",
        subset="validation",
        batch_size=32,
        seed=123,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", 
        target_size=(224,224))

Found 1101 validated image filenames belonging to 35 classes.


**Training the Model**

In [24]:
model.fit(train_gen, steps_per_epoch=20, 
                    epochs=75, validation_data=test_gen, 
                    validation_steps=20)

Epoch 1/75
20/20 [==============================] - 40s 936ms/step - loss: 5.2755 - accuracy: 0.0596 - val_loss: 4.3143 - val_accuracy: 0.0812
Epoch 2/75
20/20 [==============================] - 17s 849ms/step - loss: 3.9920 - accuracy: 0.1035 - val_loss: 4.0277 - val_accuracy: 0.1203
Epoch 3/75
20/20 [==============================] - 17s 859ms/step - loss: 3.8505 - accuracy: 0.1296 - val_loss: 3.7899 - val_accuracy: 0.1234
Epoch 4/75
20/20 [==============================] - 22s 1s/step - loss: 3.7518 - accuracy: 0.1279 - val_loss: 4.0684 - val_accuracy: 0.1047
Epoch 5/75
20/20 [==============================] - 17s 842ms/step - loss: 3.7794 - accuracy: 0.1884 - val_loss: 3.7728 - val_accuracy: 0.1203
Epoch 6/75
20/20 [==============================] - 18s 898ms/step - loss: 3.6638 - accuracy: 0.1519 - val_loss: 4.0546 - val_accuracy: 0.1031
Epoch 7/75
20/20 [==============================] - 17s 885ms/step - loss: 3.4159 - accuracy: 0.1710 - val_loss: 3.9794 - val_accuracy: 0.1422
Ep

In [ ]:
#model.fit(train_gen, steps_per_epoch=15, 
                    #epochs=16, validation_data=test_gen, 
                    #validation_steps=15)